# Google Colab Audio Recorder

This notebook demonstrates browser-based audio recording in Google Colab using JavaScript and the Web Audio API. This approach bypasses PyAudio limitations and works directly in the browser.

**🎤 Browser-Based Recording Features:**
- Direct microphone access through browser (no PyAudio needed)
- Same UI design and functionality as the original recorder
- Automatic file saving with timestamps and defect classification
- Real-time recording timer and status updates
- Compatible with Google Drive for data persistence

**📋 Instructions:**
1. **Allow Microphone Access**: Click "Allow" when your browser asks for microphone permission
2. **Select Defect Type**: Choose the appropriate classification for your recording
3. **Set Duration**: Choose recording length (1-10 seconds)
4. **Record**: Click "START RECORDING" and speak into your microphone
5. **Auto-Save**: Files are automatically saved with timestamps

In [ ]:
import os

# Clone the repository if it doesn't exist
if not os.path.exists('FrED-Predictive-Maintenance'):
    !git clone https://github.com/Ibrahimhk98/FrED-Predictive-Maintenance.git
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")

Cloning into 'FrED-Predictive-Maintenance-'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
^Cceiving objects:  22% (49/219), 102.32 MiB | 13.48 MiB/s


In [ ]:
# Install required packages (no PyAudio needed for browser recording!)
print("📦 Installing Python dependencies...")
!pip install ipywidgets numpy

print("✅ Dependencies installed - ready for browser-based recording!")

Error installing requirements: Command '['/Users/ibrahimelkhatib/miniforge3/bin/python', '-m', 'pip', 'install', '-r', '/requirements.txt']' returned non-zero exit status 1.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/requirements.txt'


In [ ]:
# Mount Google Drive for data storage
from google.colab import drive
import os

print("📁 Mounting Google Drive...")
drive.mount('/content/drive')

# Use the shared folder ID for common data storage
shared_folder_id = "1p7JVh_gL663af7pwGHQPBWmp1TiS3GPz"
data_folder = f"/content/drive/MyDrive/{shared_folder_id}"

# Create the shared folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder, exist_ok=True)
    print(f"📁 Created shared audio data folder: {data_folder}")
else:
    print(f"✅ Using shared data folder: {data_folder}")

print(f"🔗 Shared Folder ID: {shared_folder_id}")
print("🎵 Audio files will be organized by defect type automatically!")
print("👥 All students will save to the same shared location for easy access")

In [ ]:
# Verify shared folder setup and show structure
print("🔍 Verifying shared folder setup...")
print(f"📍 Shared Folder Path: {data_folder}")

# Check if the folder is accessible
if os.path.exists(data_folder):
    print("✅ Shared folder is accessible")
    
    # Show existing structure if any
    try:
        contents = os.listdir(data_folder)
        if contents:
            print(f"📂 Existing contents ({len(contents)} items):")
            for item in sorted(contents):
                item_path = os.path.join(data_folder, item)
                if os.path.isdir(item_path):
                    file_count = len([f for f in os.listdir(item_path) if f.endswith('.wav')])
                    print(f"   📁 {item}/ ({file_count} audio files)")
                else:
                    print(f"   📄 {item}")
        else:
            print("📂 Shared folder is empty - ready for new recordings")
    except Exception as e:
        print(f"⚠️ Could not list contents: {e}")
else:
    print("❌ Shared folder not accessible - creating it now...")
    os.makedirs(data_folder, exist_ok=True)

print("\n" + "="*60)
print("🎯 SHARED FOLDER CONFIGURATION")
print("="*60)
print(f"🆔 Folder ID: {shared_folder_id}")
print(f"📍 Full Path: {data_folder}")
print("👥 All student recordings will be saved here")
print("🎵 Files organized by defect type automatically")
print("="*60)

In [ ]:
# Add the repository to Python path and import the Colab audio recorder
import sys
sys.path.append('/content/FrED-Predictive-Maintenance')

# Import the new Colab-compatible audio recorder
from Preparation.Import.colab_audio_recorder import create_recorder_ui, list_colab_audio_devices

# Show available audio capabilities
print("🎤 Browser Audio Capabilities:")
for info in list_colab_audio_devices():
    print(info)

print("\n" + "="*50)
print("🚀 Launching Browser-Based Audio Recorder...")
print("="*50)

# Create and display the recorder UI
ui = create_recorder_ui(base_dir=data_folder)
display(ui)

0: Ibrahim’s iPhone Microphone
2: MacBook Pro Microphone
4: Immersed


In [ ]:
# Enhanced JavaScript audio recording setup
from IPython.display import HTML, Javascript

# Load enhanced audio recording JavaScript
display(HTML("""
<script>
// Enhanced Colab Audio Recorder with better browser compatibility
class EnhancedColabRecorder {
    constructor() {
        this.mediaRecorder = null;
        this.audioChunks = [];
        this.stream = null;
        this.isRecording = false;
        this.audioContext = null;
    }
    
    async initialize() {
        try {
            // Request microphone access with optimal settings
            const constraints = {
                audio: {
                    sampleRate: 44100,
                    channelCount: 1,
                    echoCancellation: true,
                    noiseSuppression: true,
                    autoGainControl: true
                }
            };
            
            this.stream = await navigator.mediaDevices.getUserMedia(constraints);
            
            // Create audio context for processing
            this.audioContext = new (window.AudioContext || window.webkitAudioContext)();
            
            console.log('🎤 Enhanced audio recorder initialized successfully');
            console.log(`📊 Sample rate: ${this.audioContext.sampleRate} Hz`);
            
            return true;
        } catch (error) {
            console.error('❌ Error accessing microphone:', error);
            alert('Microphone access denied. Please allow microphone access and refresh the page.');
            return false;
        }
    }
    
    startRecording() {
        if (!this.stream) {
            console.error('❌ Stream not initialized');
            return false;
        }
        
        this.audioChunks = [];
        
        // Use appropriate MIME type based on browser support
        const mimeType = this.getSupportedMimeType();
        this.mediaRecorder = new MediaRecorder(this.stream, { mimeType });
        
        this.mediaRecorder.ondataavailable = (event) => {
            if (event.data.size > 0) {
                this.audioChunks.push(event.data);
            }
        };
        
        this.mediaRecorder.onstop = () => {
            const audioBlob = new Blob(this.audioChunks, { type: mimeType });
            this.processAudio(audioBlob);
        };
        
        this.mediaRecorder.start(100); // Collect data every 100ms
        this.isRecording = true;
        
        console.log('🔴 Recording started with MIME type:', mimeType);
        return true;
    }
    
    stopRecording() {
        if (this.mediaRecorder && this.isRecording) {
            this.mediaRecorder.stop();
            this.isRecording = false;
            console.log('⏹️ Recording stopped');
            return true;
        }
        return false;
    }
    
    getSupportedMimeType() {
        const types = [
            'audio/webm;codecs=opus',
            'audio/webm',
            'audio/mp4',
            'audio/ogg;codecs=opus'
        ];
        
        for (const type of types) {
            if (MediaRecorder.isTypeSupported(type)) {
                return type;
            }
        }
        return 'audio/webm'; // fallback
    }
    
    async processAudio(audioBlob) {
        try {
            const reader = new FileReader();
            reader.onload = () => {
                const base64Audio = reader.result.split(',')[1];
                console.log(`📦 Audio processed: ${audioBlob.size} bytes`);
                
                // Send to Python
                if (typeof google !== 'undefined' && google.colab) {
                    google.colab.kernel.invokeFunction('handle_audio_data', [base64Audio], {});
                } else {
                    console.warn('⚠️ Google Colab API not available');
                }
            };
            reader.readAsDataURL(audioBlob);
        } catch (error) {
            console.error('❌ Error processing audio:', error);
        }
    }
    
    getDeviceInfo() {
        if (this.stream) {
            const track = this.stream.getAudioTracks()[0];
            const settings = track.getSettings();
            return {
                label: track.label || 'Default Microphone',
                sampleRate: settings.sampleRate || 44100,
                channelCount: settings.channelCount || 1
            };
        }
        return null;
    }
}

// Replace the global recorder with enhanced version
window.colabRecorder = new EnhancedColabRecorder();

// Initialize the enhanced recorder
window.colabRecorder.initialize().then(success => {
    if (success) {
        const deviceInfo = window.colabRecorder.getDeviceInfo();
        console.log('✅ Enhanced recorder ready!');
        if (deviceInfo) {
            console.log(`🎯 Device: ${deviceInfo.label}`);
            console.log(`📊 Config: ${deviceInfo.sampleRate}Hz, ${deviceInfo.channelCount} channel(s)`);
        }
    }
});

// Utility function to check browser compatibility
function checkBrowserCompatibility() {
    const issues = [];
    
    if (!navigator.mediaDevices || !navigator.mediaDevices.getUserMedia) {
        issues.push('getUserMedia not supported');
    }
    
    if (!window.MediaRecorder) {
        issues.push('MediaRecorder not supported');
    }
    
    if (issues.length > 0) {
        console.warn('⚠️ Browser compatibility issues:', issues);
        return false;
    }
    
    console.log('✅ Browser fully compatible with audio recording');
    return true;
}

checkBrowserCompatibility();
</script>

<style>
/* Custom styling for the audio recorder */
.audio-recorder-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.recording-indicator {
    animation: pulse 1.5s infinite;
}

@keyframes pulse {
    0% { opacity: 1; }
    50% { opacity: 0.5; }
    100% { opacity: 1; }
}

.status-success { color: #16a34a; font-weight: bold; }
.status-error { color: #dc2626; font-weight: bold; }
.status-recording { color: #d32f2f; font-weight: bold; }
</style>
"""))

print("🔧 Enhanced JavaScript audio recorder loaded!")
print("🎤 Browser compatibility checked and optimized")
print("✅ Ready for high-quality audio recording in Colab!")